# Ejercicio misceláneo: Análisis exploratorio y Feature Engineering

In [76]:
import pandas as pd
import numpy as np
import os
from scipy import stats

In [3]:
ruta_archivo = os.path.join("titanic", "train.csv")
df = pd.read_csv(os.path.join("titanic", "train.csv"), index_col='PassengerId')

In [4]:
df.head()

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
PassengerId,,,,,,,,,,,
1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


## 1. Elimine las variables/columnas 'Ticket' y 'Cabin'

In [6]:
df.drop(columns=['Cabin'], inplace=True)
df.drop(columns=['Ticket'], inplace=True)
df.head()

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Fare,Embarked
PassengerId,,,,,,,,,
1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,7.2500,S
2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,71.2833,C
3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,7.9250,S
4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,53.1000,S
5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,8.0500,S


In [15]:
df.describe(include='all')

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Fare,Embarked
count,892.000000,892.000000,892,892,715.000000,892.000000,892.000000,891.000000,890
unique,NaN,NaN,892,2,NaN,NaN,NaN,NaN,3
top,NaN,NaN,"Jonsson, Mr. Carl",male,NaN,NaN,NaN,NaN,S
freq,NaN,NaN,1,578,NaN,NaN,NaN,NaN,645
mean,0.383408,2.309417,NaN,NaN,29.742196,0.522422,0.381166,32.204208,NaN
std,0.486489,0.835923,NaN,NaN,14.561951,1.102264,0.805706,49.693429,NaN
min,0.000000,1.000000,NaN,NaN,0.420000,0.000000,0.000000,0.000000,NaN
25%,0.000000,2.000000,NaN,NaN,20.250000,0.000000,0.000000,7.910400,NaN
50%,0.000000,3.000000,NaN,NaN,28.000000,0.000000,0.000000,14.454200,NaN
75%,1.000000,3.000000,NaN,NaN,38.000000,1.000000,0.000000,31.000000,NaN


## 2. Encuentre los mejores valores para completar la variable 'Embarked' para los  pasajeros con datos faltantes

In [93]:
df[df.Embarked.isnull()]

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Fare,Embarked,Title,IsMother
PassengerId,,,,,,,,,,,


In [46]:
#Tome en cuenta que simplemente emplear el valor que más aparece casi nunca es la mejor alternativa
#Para obtener mejores resultados deberá apoyarse en los resultados de un análisis exploratorio de datos
#aplicado a las variable(s) que puedan estar relacionadas con 'Embarked'
df['Embarked'].value_counts()

S    645
C    168
Q     77
Name: Embarked, dtype: int64

In [91]:
# Obtenemos el puerto más común según la clase y el sexo
df.groupby(['Pclass', 'Sex'])['Embarked'].agg(pd.Series.mode)

Pclass  Sex   
1       female    S
        male      S
2       female    S
        male      S
3       female    S
        male      S
Name: Embarked, dtype: object

In [92]:
# Como podemos observar, todos los valores que se obtuvieron en la celda de arriba pertenecen al puerto 'S'
# Entonces, reemplazaremos los valores NaN por el puerto 'S'
df['Embarked'].fillna('S', inplace=True)
df[df.Embarked.isnull()]

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Fare,Embarked,Title,IsMother
PassengerId,,,,,,,,,,,


## 3. Complete el código de la siguiente función para extraer el título de cada pasajero

In [22]:
def extractTitle(name):
    title_mapping = {'mr' : 'Mr', 
               'mrs' : 'Mrs', 
               'miss' : 'Miss', 
               'master' : 'Master',
               'don' : 'Sir',
               'rev' : 'Sir',
               'dr' : 'Officer',
               'mme' : 'Mrs',
               'ms' : 'Mrs',
               'major' : 'Officer',
               'lady' : 'Lady',
               'sir' : 'Sir',
               'mlle' : 'Miss',
               'col' : 'Officer',
               'capt' : 'Officer',
               'the countess' : 'Lady',
               'jonkheer' : 'Sir',
               'dona' : 'Lady'
                 }
    title = name.split(', ')[1].split('.')[0].lower()
    return title_mapping[title]

In [23]:
df['Title'] =  df['Name'].map(lambda name : extractTitle(name))
df.head()

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Fare,Embarked,Title
PassengerId,,,,,,,,,,
1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,7.2500,S,Mr
2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,71.2833,C,Mrs
3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,7.9250,S,Miss
4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,53.1000,S,Mrs
5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,8.0500,S,Mr


In [24]:
df.describe(include='all')

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Fare,Embarked,Title
count,892.000000,892.000000,892,892,715.000000,892.000000,892.000000,891.000000,890,892
unique,NaN,NaN,892,2,NaN,NaN,NaN,NaN,3,7
top,NaN,NaN,"Simmons, Mr. John",male,NaN,NaN,NaN,NaN,S,Mr
freq,NaN,NaN,1,578,NaN,NaN,NaN,NaN,645,518
mean,0.383408,2.309417,NaN,NaN,29.742196,0.522422,0.381166,32.204208,NaN,NaN
std,0.486489,0.835923,NaN,NaN,14.561951,1.102264,0.805706,49.693429,NaN,NaN
min,0.000000,1.000000,NaN,NaN,0.420000,0.000000,0.000000,0.000000,NaN,NaN
25%,0.000000,2.000000,NaN,NaN,20.250000,0.000000,0.000000,7.910400,NaN,NaN
50%,0.000000,3.000000,NaN,NaN,28.000000,0.000000,0.000000,14.454200,NaN,NaN
75%,1.000000,3.000000,NaN,NaN,38.000000,1.000000,0.000000,31.000000,NaN,NaN


## 4. Encuentre los mejores valores para completar la variable 'Age' para los pasajeros con datos faltantes

Tome en cuenta que simplemente emplear el valor de tendencia central (mediana) de todo el conjunto casi nunca es la mejor alternativa
Para obtener mejores resultados deberá apoyarse en los resultados de un análisis exploratorio de datos
aplicado a las variable(s) que puedan estar relacionadas con 'Age'

In [40]:
df[df['Age'].isnull()].head()

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Fare,Embarked,Title
PassengerId,,,,,,,,,,
6,0,3,"Moran, Mr. James",male,NaN,0,0,8.4583,Q,Mr
18,1,2,"Williams, Mr. Charles Eugene",male,NaN,0,0,13.0000,S,Mr
20,1,3,"Masselmani, Mrs. Fatima",female,NaN,0,0,7.2250,C,Mrs
27,0,3,"Emir, Mr. Farred Chehab",male,NaN,0,0,7.2250,C,Mr
29,1,3,"O'Dwyer, Miss. Ellen ""Nellie""",female,NaN,0,0,7.8792,Q,Miss


In [41]:
df[df['Age'].isnull()].shape

(177, 10)

In [26]:
df['Age'].median()

28.0

In [28]:
df.corr()

,Survived,Pclass,Age,SibSp,Parch,Fare
Survived,1.000000,-0.338965,-0.079388,-0.034886,0.082010,0.257307
Pclass,-0.338965,1.000000,-0.365161,0.082599,0.017994,-0.549500
Age,-0.079388,-0.365161,1.000000,-0.308847,-0.189989,0.096067
SibSp,-0.034886,0.082599,-0.308847,1.000000,0.414985,0.159651
Parch,0.082010,0.017994,-0.189989,0.414985,1.000000,0.216225
Fare,0.257307,-0.549500,0.096067,0.159651,0.216225,1.000000


Según la matriz de correlación, *Age* tiene la correlación más alta con *Pclass*. Entonces, agruparemos los datos según este parámetro. También los agruparemos según el sexo, o sea, que los datos estarán agrupados por clase y por sexo. Porque las edades de las personas pueden variar según su sexo.

In [29]:
pasajeros_por_clase_y_sexo = df.groupby(['Pclass', 'Sex'])
mediana_age_pasajeros = pasajeros_por_clase_y_sexo['Age'].transform('median')

In [30]:
# Reemplazar los datos nulos de Age
df['Age'].fillna(mediana_age_pasajeros, inplace=True)

df[df['Age'].isnull()].head()

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Fare,Embarked,Title
PassengerId,,,,,,,,,,


## 5. Cree una nueva variable 'IsMother' (1=es madre, 0=no es madre) 

#### Responda la siguiente pregunta sustentando su respuesta con los resultados de un análisis exploratorio de datos.
#### Las madres tuvieron mayor probabilidad de sobrevivir al accidente del Titanic?

Elegimos a las madres como las mujeres que están casadas y que viajen al menos con un Parch(parientes que sean padres o hijos).

No incluímos a las mujeres con el título de Miss, para evitar incluir a las mujeres que no son madres pero que 
viajan con sus papás.

Hay que tener en cuenta que en esa época no habían muchas madres solteras (por el machismo), así que no se
pierde mucha información en caso de que una mujer sea madre y no esté casada.

In [40]:
def es_mujer_y_no_soltera_y_viaja_con_hijos(fila):
    return (fila['Parch'] > 0) & (fila['Sex'] == 'female') & (fila['Title'] != 'Miss')

def es_madre(fila):
    if es_mujer_y_no_soltera_y_viaja_con_hijos(fila):
        return 1
    return 0

df['IsMother'] = df.apply(es_madre, axis=1)
df.tail()

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Fare,Embarked,Title,IsMother
PassengerId,,,,,,,,,,,
888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,30.00,S,Miss,0
889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,21.5,1,2,23.45,S,Miss,0
890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,30.00,C,Mr,0
891,0,3,"Dooley, Mr. Patrick",male,32.0,0,0,7.75,Q,Mr,0
1044,0,3,"Storey, Mr. Thomas",male,60.5,0,0,NaN,S,Mr,0


In [41]:
df.loc[(df['IsMother'] == 1)]

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Fare,Embarked,Title,IsMother
PassengerId,,,,,,,,,,,
9,1,3,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",female,27.0,0,2,11.1333,S,Mrs,1
26,1,3,"Asplund, Mrs. Carl Oscar (Selma Augusta Emilia...",female,38.0,1,5,31.3875,S,Mrs,1
99,1,2,"Doling, Mrs. John T (Ada Julia Bone)",female,34.0,0,1,23.0000,S,Mrs,1
141,0,3,"Boulos, Mrs. Joseph (Sultana)",female,21.5,0,2,15.2458,C,Mrs,1
167,1,1,"Chibnall, Mrs. (Edith Martha Bowerman)",female,35.0,0,1,55.0000,S,Mrs,1
168,0,3,"Skoog, Mrs. William (Anna Bernhardina Karlsson)",female,45.0,1,4,27.9000,S,Mrs,1
248,1,2,"Hamalainen, Mrs. William (Anna)",female,24.0,0,2,14.5000,S,Mrs,1
252,0,3,"Strom, Mrs. Wilhelm (Elna Matilda Persson)",female,29.0,1,1,10.4625,S,Mrs,1
255,0,3,"Rosblom, Mrs. Viktor (Helena Wilhelmina)",female,41.0,0,2,20.2125,S,Mrs,1
